In [1]:
import pandas as pd
import numpy as np
import scipy
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from copy import deepcopy
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.interpolate import CubicSpline
from sklearn.metrics import mean_squared_error
import warnings
import itertools
warnings.filterwarnings("ignore")
import arviz as az
import scipy
import matplotlib.patches as patches
import statsmodels.stats.api as sms
import scipy.stats.kde as kde

# Read 16S data

In [2]:
df_16S = pd.read_excel('./16S_relative_abundance.xlsx', index_col=0)
df_16S.columns = [c.replace('/','_slash_').replace(' ','_space_').replace('[','_leftsquarebracket_').replace(']','_rightsquarebracket_').replace('-','_dash_').replace('.','_dot_').replace('(','_leftroundbracket').replace(')','_rightroundbracket_') for c in df_16S.columns]
df_16S.head()

,Absiella_dash_argi,Absiella_dash_dolichum,Acetanaerobacterium,Acetoanaerobium,Acetobacterium,Acidaminococcus_dash_intestini,Acidaminococcus_dash_sp_dot__dash_Marseille_dash_P2828,Acinetobacter,Actinobacillus,Actinobacteria,...,human_dash_gut_dash_metagenome,iron_dash_reducing_dash_bacterium_dash_enrichment_dash_culture_dash_clone_dash_HN70,marine_dash_group,marine_dash_sediment_dash_metagenome,metagenome,rumen_dash_bacterium_dash_YS2,soil_dash_metagenome,swine_dash_fecal_dash_bacterium_dash_SD_dash_Pec5,vadinBE97,wallaby_dash_gut_dash_metagenome
SampleID,,,,,,,,,,,,,,,,,,,,,
SRR6444565,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.032972,0.0,0.0,0.0,0.009201,0.0,0.0,0.0,0.0,0.0
SRR6443884,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.034554,0.0,0.0,0.0,0.007985,0.0,0.0,0.0,0.0,0.0
SRR6444382,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.032344,0.0,0.0,0.0,0.008121,0.0,0.0,0.0,0.0,0.0
SRR6444917,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.001194,0.0,0.0,0.0,0.0,0.0
SRR6443930,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


# Add pseudocount for zeros

In [3]:
for sample_id in df_16S.index:
    sample = np.array(df_16S.loc[sample_id])
    minval = np.min(sample[np.nonzero(sample)]) # minimum non-zero value
    sample[sample==0] = minval
    df_16S.loc[sample_id] = sample
df_16S.head()

,Absiella_dash_argi,Absiella_dash_dolichum,Acetanaerobacterium,Acetoanaerobium,Acetobacterium,Acidaminococcus_dash_intestini,Acidaminococcus_dash_sp_dot__dash_Marseille_dash_P2828,Acinetobacter,Actinobacillus,Actinobacteria,...,human_dash_gut_dash_metagenome,iron_dash_reducing_dash_bacterium_dash_enrichment_dash_culture_dash_clone_dash_HN70,marine_dash_group,marine_dash_sediment_dash_metagenome,metagenome,rumen_dash_bacterium_dash_YS2,soil_dash_metagenome,swine_dash_fecal_dash_bacterium_dash_SD_dash_Pec5,vadinBE97,wallaby_dash_gut_dash_metagenome
SampleID,,,,,,,,,,,,,,,,,,,,,
SRR6444565,0.000215,0.000215,0.000215,0.000215,0.000215,0.000215,0.000215,0.000215,0.000215,0.000215,...,0.032972,0.000215,0.000215,0.000215,0.009201,0.000215,0.000215,0.000215,0.000215,0.000215
SRR6443884,0.000070,0.000070,0.000070,0.000070,0.000070,0.000070,0.000070,0.000070,0.000070,0.000070,...,0.034554,0.000070,0.000070,0.000070,0.007985,0.000070,0.000070,0.000070,0.000070,0.000070
SRR6444382,0.000167,0.000167,0.000167,0.000167,0.000167,0.000167,0.000167,0.000167,0.000167,0.000167,...,0.032344,0.000167,0.000167,0.000167,0.008121,0.000167,0.000167,0.000167,0.000167,0.000167
SRR6444917,0.000199,0.000199,0.000199,0.000199,0.000199,0.000199,0.000199,0.000199,0.000199,0.000199,...,0.000199,0.000199,0.000199,0.000199,0.001194,0.000199,0.000199,0.000199,0.000199,0.000199
SRR6443930,0.000181,0.000181,0.000181,0.000181,0.000181,0.000181,0.000181,0.000181,0.000181,0.000181,...,0.000181,0.000181,0.000181,0.000181,0.000181,0.000181,0.000181,0.000181,0.000181,0.000181


# Read metadata for maize starch and control

In [4]:
df_meta = pd.read_csv('./metadata.txt', sep='\t', index_col=0)
df_meta = df_meta[['Group','Subject_ID','status']]
df_meta = df_meta[(df_meta.Group=='himaize') | (df_meta.Group=='accessible')]
df_meta['Day'] = [0 if s=='before' else 21 for s in df_meta.status]
df_meta['Diet'] = [1 if g=='himaize' else 0 for g in df_meta['Group']]
df_meta = df_meta.drop('Group', axis=1)
df_meta.head()

,Subject_ID,status,Day,Diet
#SampleID,,,,
SRR6444565,U222,before,0,0
SRR6443884,U222,before,0,0
SRR6444382,U222,before,0,0
SRR6444917,U225,before,0,0
SRR6443930,U225,before,0,0


# Select the 20 most abundant taxa

In [5]:
topX = 20
df_16S_T = df_16S.loc[df_meta.index].T
df_16S_T['mean'] = df_16S_T.mean(axis=1)
df_16S_T = df_16S_T.sort_values(by=['mean'],axis=0,ascending=False)
df_16S_T = df_16S_T.drop('mean', axis=1)
df_16S_topX = df_16S_T.iloc[0:topX].T
df_16S_topX.head()

,Bacteroides,Faecalibacterium,Lachnospiraceae,Prevotella,Trichuris_dash_trichiura_dash__leftroundbrackethuman_dash_whipworm_rightroundbracket_,Ruminococcus_dash_sp_dot__dash_N15_dot_MGS_dash_57,Alistipes,Parabacteroides,Bacteroides_dash_plebeius_dash_DSM_dash_17135,Blautia,gut_dash_metagenome,Parasutterella_dash_excrementihominis,Roseburia,Ruminococcus,Anaerostipes,Bacteroides_dash_coprocola,metagenome,Lachnospira,Fusicatenibacter,Subdoligranulum
#SampleID,,,,,,,,,,,,,,,,,,,,
SRR6444565,0.197580,0.136827,0.024022,0.000215,0.000215,0.037984,0.028962,0.007446,0.114631,0.018580,0.036051,0.000215,0.027852,0.006623,0.006158,0.000215,0.009201,0.010847,0.003508,0.005907
SRR6443884,0.243131,0.099128,0.017050,0.000070,0.000070,0.037064,0.032008,0.006764,0.125837,0.014191,0.034100,0.000070,0.022873,0.005335,0.005230,0.000070,0.007985,0.013982,0.002789,0.004079
SRR6444382,0.214061,0.110743,0.028395,0.000167,0.000167,0.037600,0.031565,0.006981,0.115360,0.020525,0.032734,0.000167,0.029730,0.006869,0.007231,0.000167,0.008121,0.011263,0.003560,0.005006
SRR6444917,0.578520,0.063804,0.117987,0.000199,0.000199,0.000630,0.037393,0.003550,0.000199,0.014168,0.000630,0.026942,0.000199,0.003782,0.000299,0.000199,0.001194,0.000199,0.000199,0.000199
SRR6443930,0.610679,0.055143,0.050196,0.000181,0.000181,0.000181,0.079698,0.004404,0.000181,0.006154,0.000181,0.023409,0.000181,0.000181,0.000181,0.000181,0.000181,0.000181,0.000181,0.000181


In [6]:
list(df_16S_topX.columns)

['Bacteroides',
 'Faecalibacterium',
 'Lachnospiraceae',
 'Prevotella',
 'Trichuris_dash_trichiura_dash__leftroundbrackethuman_dash_whipworm_rightroundbracket_',
 'Ruminococcus_dash_sp_dot__dash_N15_dot_MGS_dash_57',
 'Alistipes',
 'Parabacteroides',
 'Bacteroides_dash_plebeius_dash_DSM_dash_17135',
 'Blautia',
 'gut_dash_metagenome',
 'Parasutterella_dash_excrementihominis',
 'Roseburia',
 'Ruminococcus',
 'Anaerostipes',
 'Bacteroides_dash_coprocola',
 'metagenome',
 'Lachnospira',
 'Fusicatenibacter',
 'Subdoligranulum']

# Normalize max value of 16S data to 1

In [7]:
bacterial_taxa = list(df_16S_topX.columns)
df_16S_topX_w_meta = pd.merge(df_meta, df_16S_topX/df_16S_topX.max().max(), left_index=True, right_index=True, how='inner')
df_16S_topX_w_meta = df_16S_topX_w_meta.groupby(['Subject_ID','status']).agg(np.mean).reset_index()
df_16S_topX_w_meta.head()

,Subject_ID,status,Day,Diet,Bacteroides,Faecalibacterium,Lachnospiraceae,Prevotella,Trichuris_dash_trichiura_dash__leftroundbrackethuman_dash_whipworm_rightroundbracket_,Ruminococcus_dash_sp_dot__dash_N15_dot_MGS_dash_57,...,gut_dash_metagenome,Parasutterella_dash_excrementihominis,Roseburia,Ruminococcus,Anaerostipes,Bacteroides_dash_coprocola,metagenome,Lachnospira,Fusicatenibacter,Subdoligranulum
0,U043,before,0,1,0.566560,0.181597,0.014493,0.000411,0.000411,0.048429,...,0.003573,0.000411,0.030010,0.000411,0.015531,0.000411,0.000411,0.000411,0.000411,0.000411
1,U043,during,21,1,0.531151,0.161992,0.016003,0.004825,0.002562,0.068180,...,0.002982,0.000185,0.054666,0.000185,0.010017,0.000185,0.000241,0.000185,0.000185,0.000185
2,U044,before,0,1,0.067995,0.090265,0.034407,0.146068,0.008834,0.000273,...,0.076503,0.004303,0.003725,0.000220,0.013594,0.000220,0.003754,0.003344,0.012889,0.015563
3,U044,during,21,1,0.053179,0.064275,0.078139,0.153087,0.010942,0.000443,...,0.094488,0.004591,0.003187,0.000151,0.012505,0.000151,0.012696,0.003020,0.009586,0.013215
4,U045,before,0,1,0.094715,0.093115,0.057557,0.136424,0.132611,0.054321,...,0.006494,0.000201,0.000201,0.063839,0.025264,0.000201,0.034321,0.000201,0.021834,0.003986


# Calculate log-derivatives (dlog(B)/dt)

In [8]:
df_deriv = deepcopy(df_16S_topX_w_meta)
for curr_sub in set(df_deriv.Subject_ID):
    curr_df = df_deriv[df_deriv.Subject_ID==curr_sub].sort_values(by='Day')
    for taxon in bacterial_taxa:
        xdata = np.array(curr_df['Day'])
        ydata = np.array(curr_df[taxon])
        cs = CubicSpline(xdata, ydata)
        csd1 = cs.derivative(nu=1)
        ydata_d1 = csd1(xdata)
        df_deriv.loc[df_deriv.Subject_ID==curr_sub, taxon] = ydata_d1
df_deriv.head()

,Subject_ID,status,Day,Diet,Bacteroides,Faecalibacterium,Lachnospiraceae,Prevotella,Trichuris_dash_trichiura_dash__leftroundbrackethuman_dash_whipworm_rightroundbracket_,Ruminococcus_dash_sp_dot__dash_N15_dot_MGS_dash_57,...,gut_dash_metagenome,Parasutterella_dash_excrementihominis,Roseburia,Ruminococcus,Anaerostipes,Bacteroides_dash_coprocola,metagenome,Lachnospira,Fusicatenibacter,Subdoligranulum
0,U043,before,0,1,-0.001686,-0.000934,0.000072,0.000210,0.000102,0.000941,...,-0.000028,-0.000011,0.001174,-0.000011,-0.000263,-0.000011,-0.000008,-0.000011,-0.000011,-0.000011
1,U043,during,21,1,-0.001686,-0.000934,0.000072,0.000210,0.000102,0.000941,...,-0.000028,-0.000011,0.001174,-0.000011,-0.000263,-0.000011,-0.000008,-0.000011,-0.000011,-0.000011
2,U044,before,0,1,-0.000706,-0.001238,0.002082,0.000334,0.000100,0.000008,...,0.000856,0.000014,-0.000026,-0.000003,-0.000052,-0.000003,0.000426,-0.000015,-0.000157,-0.000112
3,U044,during,21,1,-0.000706,-0.001238,0.002082,0.000334,0.000100,0.000008,...,0.000856,0.000014,-0.000026,-0.000003,-0.000052,-0.000003,0.000426,-0.000015,-0.000157,-0.000112
4,U045,before,0,1,-0.000367,0.000556,-0.000868,0.000506,0.000669,0.001538,...,-0.000109,-0.000004,0.000067,0.000737,-0.000446,-0.000004,-0.000621,-0.000004,-0.000488,-0.000043


# Construct regression matrix

In [9]:
# Ymat should be samples by bacteria
Ymat = df_deriv[bacterial_taxa].values
Ymat = Ymat.flatten(order='F')
Ymat = StandardScaler().fit_transform(Ymat.reshape(-1,1)).reshape(1,-1)[0] # standardize
Ymat

array([-1.24868729, -1.24868729, -0.5259887 , ...,  0.12887521,
        0.35415503,  0.35415503])

In [10]:
len(Ymat)

3280

In [11]:
Xmat = np.zeros(shape=(topX*len(df_deriv.index), (topX+2)*topX))
for k in np.arange(topX):
    Xmat[k*len(df_deriv.index):(k+1)*len(df_deriv.index),k*(topX+2)] = 1
    Xmat[k*len(df_deriv.index):(k+1)*len(df_deriv.index),k*(topX+2)+1] = df_deriv.Diet.values
    Xmat[k*len(df_deriv.index):(k+1)*len(df_deriv.index),k*(topX+2)+2:(k+1)*(topX+2)] = df_16S_topX_w_meta[bacterial_taxa].values
Xmat

array([[1.00000000e+00, 1.00000000e+00, 5.66560062e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 1.00000000e+00, 5.31150643e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 1.00000000e+00, 6.79946621e-02, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        1.29988210e-03, 4.69189295e-03, 3.10771634e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        2.84350475e-03, 5.87330579e-04, 1.78306210e-02],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        4.29800958e-03, 4.90891772e-03, 2.80945526e-02]])

# Write stan program

## write data

In [24]:
json_str = '{\n"N" : %d,\n'%(len(Ymat))

# y variable
json_str += '\"dlogX\" : [%s],\n'%(','.join(list(Ymat.astype(str))))

# x variable
for k1,c1 in enumerate(bacterial_taxa):
    # growth rate
    json_str += '\"growth_rate_%s\" : [%s],\n'%(c1,','.join(list(Xmat[:,k1*(topX+2)].astype(str))))
    # diet response
    json_str += '\"inulin_response_%s\" : [%s],\n'%(c1,','.join(list(Xmat[:,k1*(topX+2)+1].astype(str))))
    # bacterial interactions
    for k2,c2 in enumerate(bacterial_taxa):
        v = list(Xmat[:,k1*(topX+2)+2+k2].astype(str))
        json_str += '\"pairwise_interaction_%s_%s\" : [%s]'%(c1,c2,','.join(v))
        if c1 == bacterial_taxa[-1] and c2 == bacterial_taxa[-1]:
            json_str += '\n}'
        else:
            json_str += ',\n'
#print(json_str)

text_file = open("/Users/liaoc/Documents/cmdstan-2.24.1/examples/mice_scfa_2/mice_scfa_2.data.json", "w")
text_file.write("%s" % json_str)
text_file.close()

## write model

In [25]:
# data block
model_str = 'data {\n'
model_str += '\tint<lower=0> N;\n'
model_str += '\tvector[N] dlogX;\n'
for c1 in bacterial_taxa:
    model_str += '\tvector[N] growth_rate_%s;\n'%(c1)
    model_str += '\tvector[N] inulin_response_%s;\n'%(c1)
    for c2 in bacterial_taxa:
        model_str += '\tvector[N] pairwise_interaction_%s_%s;\n'%(c1,c2)
model_str += '}\n'

# parameter block
model_str += 'parameters {\n\treal<lower=0,upper=1> sigma;\n'
for c1 in bacterial_taxa:
    model_str += '\treal alpha__%s;\n'%(c1) # growth rate
    model_str += '\treal epsilon__%s;\n'%(c1) # inulin response
    for c2 in bacterial_taxa:
        model_str += '\treal beta__%s_%s;\n'%(c1,c2)
model_str += '}\n'       
        
# model block
model_str += 'model {\n\tsigma ~ uniform(0,1);\n'
for c1 in bacterial_taxa:
    model_str += '\talpha__%s ~ normal(0,1);\n'%(c1) # growth rate
    model_str += '\tepsilon__%s ~ normal(0,1);\n'%(c1) # inulin response
    for c2 in bacterial_taxa:
        model_str += '\tbeta__%s_%s ~ normal(0,1);\n'%(c1,c2)
model_str += '\tdlogX ~ normal('
for c1 in bacterial_taxa:
    model_str += 'alpha__%s*growth_rate_%s+'%(c1,c1) # growth rate
    model_str += 'epsilon__%s*inulin_response_%s+'%(c1,c1) # inulin response
    for c2 in bacterial_taxa:
        if c1 == bacterial_taxa[-1] and c2 == bacterial_taxa[-1]:
            model_str += 'beta__%s_%s*pairwise_interaction_%s_%s'%(c1,c2,c1,c2)
        else:
            model_str += 'beta__%s_%s*pairwise_interaction_%s_%s+'%(c1,c2,c1,c2)
model_str += ', sigma);\n}'
#print(model_str)

text_file = open("/Users/liaoc/Documents/cmdstan-2.24.1/examples/mice_scfa_2/mice_scfa_2.stan", "w")
text_file.write("%s" % model_str)
text_file.close()

# Plot stan output

In [12]:
def hpd_grid(sample, alpha=0.05, roundto=2):
    """Calculate highest posterior density (HPD) of array for given alpha. 
    The HPD is the minimum width Bayesian credible interval (BCI). 
    The function works for multimodal distributions, returning more than one mode
    Parameters
    ----------
    
    sample : Numpy array or python list
        An array containing MCMC samples
    alpha : float
        Desired probability of type I error (defaults to 0.05)
    roundto: integer
        Number of digits after the decimal point for the results
    Returns
    ----------
    hpd: array with the lower 
          
    """
    sample = np.asarray(sample)
    sample = sample[~np.isnan(sample)]
    # get upper and lower bounds
    l = np.min(sample)
    u = np.max(sample)
    density = kde.gaussian_kde(sample)
    x = np.linspace(l, u, 2000)
    y = density.evaluate(x)
    #y = density.evaluate(x, l, u) waitting for PR to be accepted
    xy_zipped = zip(x, y/np.sum(y))
    xy = sorted(xy_zipped, key=lambda x: x[1], reverse=True)
    xy_cum_sum = 0
    hdv = []
    for val in xy:
        xy_cum_sum += val[1]
        hdv.append(val[0])
        if xy_cum_sum >= (1-alpha):
            break
    hdv.sort()
    diff = (u-l)/20  # differences of 5%
    hpd = []
    hpd.append(round(min(hdv), roundto))
    for i in range(1, len(hdv)):
        if hdv[i]-hdv[i-1] >= diff:
            hpd.append(round(hdv[i-1], roundto))
            hpd.append(round(hdv[i], roundto))
    hpd.append(round(max(hdv), roundto))
    ite = iter(hpd)
    hpd = list(zip(ite, ite))
    modes = []
    for value in hpd:
        x_hpd = x[(x > value[0]) & (x < value[1])]
        y_hpd = y[(x > value[0]) & (x < value[1])]
        modes.append(round(x_hpd[np.argmax(y_hpd)], roundto))
    return hpd, x, y, modes

In [13]:
fit = az.from_cmdstan(["output_%d_rs.csv"%(i) for i in np.arange(1,4)])

In [17]:
lines = []
    
# basal growth rate
for c in bacterial_taxa:
    var = 'alpha__%s'%(c)
    data = []
    for i in np.arange(0,3):
        data.extend(list(fit.posterior[var][i].values))
    hpd_mu, x_mu, y_mu, modes_mu = hpd_grid(data)
    assert len(hpd_mu) == 1
    (x0, x1) = hpd_mu[0]
    lines.append(['basal_growth_rate', c, x0, x1, (x0+x1)/2, np.mean(data)/np.std(data), x0*x1>0])
    
# inulin response
for c in bacterial_taxa:
    var = 'epsilon__%s'%(c)
    data = []
    for i in np.arange(0,3):
        data.extend(list(fit.posterior[var][i].values))
    hpd_mu, x_mu, y_mu, modes_mu = hpd_grid(data)
    assert len(hpd_mu) == 1
    (x0, x1) = hpd_mu[0]
    lines.append(['rs_response', c, x0, x1, (x0+x1)/2, np.mean(data)/np.std(data), x0*x1>0])

# pairwise interactions
for c1 in bacterial_taxa:
    for c2 in bacterial_taxa:
        var = 'beta__%s_%s'%(c1,c2)
        data = []
        for i in np.arange(0,3):
            data.extend(list(fit.posterior[var][i].values))
        hpd_mu, x_mu, y_mu, modes_mu = hpd_grid(data)
        assert len(hpd_mu) == 1
        (x0, x1) = hpd_mu[0]
        lines.append(['pairwise_interaction', (c1,c2), x0, x1, (x0+x1)/2, np.mean(data)/np.std(data), x0*x1>0])
    
df_stan_output_summary = pd.DataFrame(lines, columns = ['Type','Taxa','Left','Right','Middle','SNR','Significant'])
df_stan_output_summary.head()

,Type,Taxa,Left,Right,Middle,SNR,Significant
0,basal_growth_rate,Bacteroides,-1.14,-0.11,-0.625,-2.409779,True
1,basal_growth_rate,Faecalibacterium,-0.66,0.36,-0.150,-0.596851,False
2,basal_growth_rate,Lachnospiraceae,-0.42,0.61,0.095,0.360303,False
3,basal_growth_rate,Prevotella,-0.39,0.63,0.120,0.454670,False
4,basal_growth_rate,Trichuris_dash_trichiura_dash__leftroundbracke...,-0.68,0.33,-0.175,-0.646623,False


In [18]:
df_stan_output_summary.to_excel('bayesian_regression_summary_top20_rs.xlsx')

# Plot

In [19]:
df_stan_output_summary = pd.read_excel('bayesian_regression_summary_top20_rs.xlsx', index_col=0)
df_stan_output_summary = df_stan_output_summary[df_stan_output_summary.Significant==True]
df_stan_output_summary

,Type,Taxa,Left,Right,Middle,SNR,Significant
0,basal_growth_rate,Bacteroides,-1.14,-0.11,-0.625,-2.409779,True
5,basal_growth_rate,Ruminococcus_dash_sp_dot__dash_N15_dot_MGS_das...,0.10,1.13,0.615,2.367750,True
8,basal_growth_rate,Bacteroides_dash_plebeius_dash_DSM_dash_17135,-1.05,-0.02,-0.535,-2.071025,True
25,rs_response,Ruminococcus_dash_sp_dot__dash_N15_dot_MGS_das...,0.22,0.81,0.515,3.448608,True
40,pairwise_interaction,"('Bacteroides', 'Bacteroides')",1.07,2.51,1.790,4.897480,True
41,pairwise_interaction,"('Bacteroides', 'Faecalibacterium')",0.51,3.99,2.250,2.558270,True
42,pairwise_interaction,"('Bacteroides', 'Lachnospiraceae')",-4.60,-1.04,-2.820,-3.107074,True
43,pairwise_interaction,"('Bacteroides', 'Prevotella')",-2.47,-0.20,-1.335,-2.315486,True
44,pairwise_interaction,"('Bacteroides', 'Trichuris_dash_trichiura_dash...",1.42,3.85,2.635,4.293403,True
45,pairwise_interaction,"('Bacteroides', 'Ruminococcus_dash_sp_dot__das...",-4.96,-1.38,-3.170,-3.496771,True
